# Overview

**Context** : 

Data were extracted from images that were taken from genuine and forged banknote-like specimens. The final images have 400×400 pixels. Due to the object lens and distance to the investigated object gray-scale, pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.

**Content** :

The first four columns are features and the last column is label.

 - Variance of Wavelet Transformed Image
 - Skewness of Wavelet Transformed Image
 - Curtosis of Wavelet Transformed Image
 - Entropy of Image
 
**Output Feature / Label :** 1 - Real ; 0 - Fake

# Let's Work 

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [6]:
# Reading the dataset

data = pd.read_csv('../input/bank_note_data.csv')

In [7]:
data.info()

In [8]:
data.head()

In [9]:
print("Total Columns : ", len(data.columns))

In [10]:
# Total Labels
data['Class'].value_counts()

## Input and Output Data

In [11]:
X = data[data.columns[0:4]].values
y = data[data.columns[4]].values

In [12]:
print(X.shape)

## Using OneHotEncoder

In [13]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [14]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
Y = one_hot_encode(y)

In [15]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

Y = one_hot_encode(y)

In [42]:
print(Y[0])
print(Y.shape)

## Training and Testing Data

As our output column has labels arranged together, so we first shuffle our data

In [17]:
X,Y = shuffle (X, Y, random_state = 10)

In [18]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size =0.20, random_state=20)

In [22]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

# Neural Net Model

In [25]:
learning_rate = 0.1
training_epochs = 1000

In [26]:
n_dim = X.shape[1]
print("n_dim = ", n_dim)

n_class = 2

In [27]:
cost_history = np.empty(shape=[1],dtype=float)

## 4 layer Dense Neural Networks

In [29]:
n_hidden_1 = 4
n_hidden_2 = 4
n_hidden_3 = 4
n_hidden_4 = 4 

In [30]:
x = tf.placeholder(tf.float32, [None, n_dim])
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [None, n_class])

In [31]:
# Define the model
def multilayer_perceptron(x, weights, biases):
 
    # Hidden layer with RELU activationsd
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
 
    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
 
    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
 
    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.sigmoid(layer_4)
 
    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer
 

In [32]:
# Define the weights for each layers
 
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, n_class]))
}

In [33]:
# Define the bias for each layers

biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

In [34]:
init = tf.global_variables_initializer()

In [35]:
# Calling model
y = multilayer_perceptron(x, weights, biases)

## Cost Function

In [36]:
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)


## Initializing Session

In [37]:
sess = tf.Session()
sess.run(init) 

In [38]:
mse_history = []
accuracy_history = []

## Training Model

In [39]:
for epoch in range(training_epochs):
    sess.run(training_step, feed_dict = {x: train_x, y_: train_y})
    cost = sess.run(cost_function, feed_dict={x: train_x, y_: train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    pred_y = sess.run(y, feed_dict = {x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))
    accuracy_history.append(accuracy)
    
    if epoch % 100 == 0:
        print('epoch : ', epoch, ' ; ', 'cost: ', cost, " ; MSE: ", mse_, "- Train Accuracy: ", accuracy )
 

## Cost Graph

In [43]:
plt.plot(range(len(cost_history)), cost_history)
plt.xlabel('Epochs ')
plt.ylabel('Cost_History ')
plt.title("Cost per Epoch Graph")
plt.xlim(0, training_epochs + 10)
plt.ylim(0, np.max(cost_history))
plt.rcParams['figure.figsize'] = [12, 5]
plt.show()

# Accuracy Graph

In [47]:
plt.plot(range(len(accuracy_history)), accuracy_history)
plt.xlabel('Epochs ')
plt.ylabel('Accuracy_History ')
plt.title("Cost per Epoch Graph")
plt.xlim(0, training_epochs + 10)
plt.ylim(0.4, np.max(accuracy_history)+0.2)
plt.rcParams['figure.figsize'] = [12, 5]
plt.show()

### Accuracy

In [48]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))

### Mean Square Error

In [49]:
pred_y = sess.run(y, feed_dict={x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))

#  Classification using Sklearn

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Splitting Training and Testing Data

In [25]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size =0.20, random_state=20)

In [43]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

# Classification : KNN

In [46]:
# train
knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric='minkowski')
knn_classifier.fit(xtrain, ytrain)

In [27]:
# Predicting 
y_pred1 = knn_classifier.predict(xtest)

In [36]:
print("Labels : ", accuracy_score(ytest, y_pred1, normalize = False))
print("Total Labels : ", len(ytest))

.

Thanks for having a look. If u like the Deep Neural Network approach, Pls give a LIKE to my kernel :)